In [7]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as multi
from tqdm import tqdm
from tabulate import tabulate

## Metadata

In [47]:
metadata_dict = {}
metadata = pd.read_csv('Metadata_HB/HB_joint_METADATA.tsv',sep='\t')
lst = []
for i in metadata['type']:
    if str(i).startswith('Hepatoblastoma'):
        lst.append('case')
    elif str(i).startswith('Normal'):
        lst.append('control')
    else:
        lst.append('other')
metadata['class'] = lst
metadata_dict = pd.Series(metadata['class'].values,index=metadata['sample']).to_dict()

## Statistic Tests

In [65]:
def stats_tests(data):
    # Independent Unequal Variance T-Test per gene
    tstatistic = []
    pvalue = []
    effsize = []
    mean_case = []
    mean_control = []
    std_case = []
    std_control = []
    for idx in tqdm(data.index):

        case = data['case'].loc[idx].tolist()
        control = data['control'].loc[idx].tolist()

        t = stats.ttest_ind(case, control, equal_var = False)
        tstatistic.append(t[0])
        pvalue.append(t[1])

        # Cohen's d effect size (i.e., standardized mean difference)
        # d=0.2 small; d=0.5 medium; d=0.8 large

        es = (np.mean(case) - np.mean(control)) / np.std(case+control)
        effsize.append(es)
        mean_case.append(np.mean(case))
        mean_control.append(np.mean(control))
        std_case.append(np.std(case))
        std_control.append(np.std(control))

    data['mean_case'] = mean_case
    data['mean_control'] = mean_case
    data['std_case'] = std_case
    data['std_control'] = std_control
    data['t-statistic'] = tstatistic
    data['p-value'] = pvalue
    data['effect size'] = effsize

## Ranganathan (2016) - GSE89775

In [66]:
GSE89775 = pd.read_csv('Matrices_HB/GSE89775_matrix.txt', sep=";", index_col=0)
GSE89775.columns = GSE89775.columns.map(metadata_dict)
GSE89775.head()

,control,control,control,case,case,case,case,case,case,case,case,case,case
5S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A1BG,12.945060,12.834613,13.572898,4.314547,11.379509,9.520326,13.544350,10.913141,14.722116,11.532927,14.127765,12.967832,9.786138
A1CF,11.364671,11.415502,11.908550,3.252039,10.259515,9.299494,9.304668,6.803130,9.686471,7.823505,9.040708,8.642276,9.435681
A2M,16.843844,16.728888,17.173702,13.094049,15.938659,13.290129,15.869329,12.050557,15.501356,13.958533,15.932645,14.197312,13.450614
A2M-AS1,3.889575,6.061951,6.393288,5.850344,3.929974,5.449400,5.272164,4.776655,4.444222,3.747714,4.255318,4.485382,0.000000


In [67]:
stats_tests(GSE89775)

  0%|          | 0/45000 [00:00<?, ?it/s]<ipython-input-65-be3667068872>:22: RuntimeWarning: invalid value encountered in double_scalars
  es = (np.mean(case) - np.mean(control)) / np.std(case+control)
100%|██████████| 45000/45000 [01:23<00:00, 542.04it/s]


higher effect-size:
           t-statistic       p-value  effect size
CSNK1D      10.460063  2.223112e-03     2.283285
MED15        7.821516  1.087733e-02     2.283865
SUV420H2    16.074888  2.409908e-06     2.285672
MYPOP       20.940203  3.675104e-10     2.293405
VAMP2       14.487254  1.153407e-04     2.297476
ARHGAP33    10.128217  4.380738e-03     2.302523
EXD3        17.460260  1.626057e-05     2.312186
PCIF1       14.377365  7.886138e-04     2.322229
MED25       27.626151  3.136056e-11     2.328587
CAPS        28.488391  2.684668e-11     2.331648 
lower effect-size:
                t-statistic   p-value  effect size
RNU6-1054P      -22.047842  0.002051    -2.367142
AC008269.2      -39.963739  0.000013    -2.365151
RNU6-49P        -18.755730  0.002831    -2.364742
RP11-570H19.2   -18.755730  0.002831    -2.364742
AE000662.92     -18.755730  0.002831    -2.364742
RP11-472F19.1   -18.755730  0.002831    -2.364742
RP11-248E9.7    -18.755730  0.002831    -2.364742
RNU2-63P        -17

In [83]:
# saving and viewing results
GSE89775_stats = GSE89775[["t-statistic","p-value","effect size"]]
GSE89775_stats = GSE89775_stats.dropna(axis=0, subset=["effect size"])
GSE89775_stats = GSE89775_stats.sort_values(by="effect size")
print('higher effect-size:\n',
      GSE89775_stats.tail(10),
      '\n\nlower effect-size:\n',
      GSE89775_stats.head(10))

higher effect-size:
           t-statistic       p-value  effect size
CSNK1D      10.460063  2.223112e-03     2.283285
MED15        7.821516  1.087733e-02     2.283865
SUV420H2    16.074888  2.409908e-06     2.285672
MYPOP       20.940203  3.675104e-10     2.293405
VAMP2       14.487254  1.153407e-04     2.297476
ARHGAP33    10.128217  4.380738e-03     2.302523
EXD3        17.460260  1.626057e-05     2.312186
PCIF1       14.377365  7.886138e-04     2.322229
MED25       27.626151  3.136056e-11     2.328587
CAPS        28.488391  2.684668e-11     2.331648 

lower effect-size:
                t-statistic   p-value  effect size
RNU6-1054P      -22.047842  0.002051    -2.367142
AC008269.2      -39.963739  0.000013    -2.365151
RNU6-49P        -18.755730  0.002831    -2.364742
RP11-570H19.2   -18.755730  0.002831    -2.364742
AE000662.92     -18.755730  0.002831    -2.364742
RP11-472F19.1   -18.755730  0.002831    -2.364742
RP11-248E9.7    -18.755730  0.002831    -2.364742
RNU2-63P        -1

## Hooks (2018) - GSE104766

In [90]:
GSE104766 = pd.read_csv('Matrices_HB/GSE104766_matrix.txt',sep=';', index_col=0)
GSE104766.columns = GSE104766.columns.map(metadata_dict);
GSE104766.head()

,control,case,control,case,control,case,control,case,control,case,...,control,case,case,case,control,case,control,case,case,case
symbol,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.000000,0.750000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.5
5_8S_rRNA,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.833333,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.166667,0.166667,0.166667,0.000000,0.000000,0.000000,0.0
7SK,0.428571,0.142857,0.428571,0.0,0.285714,1.0,1.000000,0.285714,2.714286,0.428571,...,0.714286,0.0,0.0,0.857143,0.142857,0.000000,0.571429,0.285714,0.142857,0.0
A1BG,614.000000,642.000000,253.000000,117.0,3021.000000,2729.0,859.000000,992.000000,1034.000000,1527.000000,...,1016.000000,2309.0,1019.0,1514.000000,738.000000,700.000000,458.000000,330.000000,72.000000,1.0
A1BG-AS1,40.000000,46.000000,29.000000,5.0,131.000000,149.0,33.000000,47.000000,47.000000,52.000000,...,34.000000,59.0,36.0,34.000000,20.000000,24.000000,33.000000,19.000000,0.000000,0.0


In [91]:
stats_tests(GSE104766)

  0%|          | 13/39331 [00:00<05:12, 125.77it/s]<ipython-input-65-be3667068872>:22: RuntimeWarning: invalid value encountered in double_scalars
  es = (np.mean(case) - np.mean(control)) / np.std(case+control)
100%|██████████| 39331/39331 [03:41<00:00, 177.33it/s]


higher effect-size:
          t-statistic       p-value  effect size
symbol                                         
MLLT6       6.879935  5.547744e-09     1.320641
SLC6A9      7.072303  3.955181e-08     1.326720
TRIB2       7.071840  1.558177e-08     1.330700
ZBTB40      7.157834  4.139357e-09     1.344760
CRTAP       7.296785  4.668115e-09     1.355939
ZNF775      7.312788  5.559434e-09     1.356545
FAM3B       7.523684  9.938717e-09     1.372542
ITGA6       7.625050  3.449875e-09     1.384952
ANKRD65     7.847013  2.206638e-09     1.404747
IGF2BP2     8.850698  5.239498e-12     1.496530 
lower effect-size:
            t-statistic       p-value  effect size
symbol                                           
HP           -9.846440  3.481976e-12    -1.585816
C3P1         -9.635238  7.414044e-12    -1.573208
ADORA2BP1    -9.453222  2.389087e-11    -1.563059
C9           -9.434889  3.698900e-11    -1.562709
XDH          -9.486416  4.582228e-12    -1.561773
GLYATL1      -9.153265  6.396167

In [92]:
# saving and viewing results
GSE104766_stats = GSE104766[["t-statistic","p-value","effect size"]]
GSE104766_stats = GSE104766_stats.dropna(axis=0, subset=["effect size"])
GSE104766_stats = GSE104766_stats.sort_values(by="effect size")
print('higher effect-size:\n',
      GSE104766_stats.tail(10),
      '\n\nlower effect-size:\n',
      GSE104766_stats.head(10))

higher effect-size:
          t-statistic       p-value  effect size
symbol                                         
MLLT6       6.879935  5.547744e-09     1.320641
SLC6A9      7.072303  3.955181e-08     1.326720
TRIB2       7.071840  1.558177e-08     1.330700
ZBTB40      7.157834  4.139357e-09     1.344760
CRTAP       7.296785  4.668115e-09     1.355939
ZNF775      7.312788  5.559434e-09     1.356545
FAM3B       7.523684  9.938717e-09     1.372542
ITGA6       7.625050  3.449875e-09     1.384952
ANKRD65     7.847013  2.206638e-09     1.404747
IGF2BP2     8.850698  5.239498e-12     1.496530 

lower effect-size:
            t-statistic       p-value  effect size
symbol                                           
HP           -9.846440  3.481976e-12    -1.585816
C3P1         -9.635238  7.414044e-12    -1.573208
ADORA2BP1    -9.453222  2.389087e-11    -1.563059
C9           -9.434889  3.698900e-11    -1.562709
XDH          -9.486416  4.582228e-12    -1.561773
GLYATL1      -9.153265  6.39616

## Wagner (2020) - GSE151347

In [93]:
GSE151347 = pd.read_csv('Matrices_HB/GSE151347_matrix.txt', sep=';')
GSE151347.columns = GSE151347.columns.map(metadata_dict)
GSE151347.head()

,NaN,control,control,control,control,control,control,control,control,control,...,case,case,case,case,case,case,case,case,case,case
0,5S_rRNA,41,53,29,46,38,42,63,41,56,...,56,46,56,159,66,49,99,341,51,101
1,5_8S_rRNA,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
2,6M1-18,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,2,0,1
3,7M1-2,1,1,0,1,0,0,2,0,0,...,1,0,1,0,0,0,0,1,0,0
4,7SK,228,192,125,136,175,167,187,145,103,...,73,66,145,217,125,36,94,103,58,123


In [94]:
stats_tests(GSE151347)

  0%|          | 0/28516 [00:00<?, ?it/s]<ipython-input-65-be3667068872>:22: RuntimeWarning: invalid value encountered in double_scalars
  es = (np.mean(case) - np.mean(control)) / np.std(case+control)
100%|██████████| 28516/28516 [00:58<00:00, 490.46it/s]


higher effect-size:
        t-statistic       p-value  effect size
9313      7.486681  1.160394e-05     1.716993
27907     7.550029  8.416382e-07     1.720778
21825     7.641057  1.612375e-05     1.726096
9289      7.699302  3.582986e-06     1.729424
12728     8.116543  4.187021e-06     1.751699
24831     8.492090  1.242135e-06     1.769612
6644      8.495601  9.495677e-07     1.769771
104       8.599307  2.068259e-06     1.774392
10870     8.859749  2.367085e-08     1.785435
23491     9.438096  1.990221e-06     1.807368 
lower effect-size:
        t-statistic       p-value  effect size
12406   -10.851753  5.371941e-08    -1.849130
12397   -10.669898  1.970289e-08    -1.844533
9932    -10.591128  4.137439e-07    -1.842479
20677    -9.105865  1.719878e-06    -1.795180
4803     -8.939265  2.624318e-06    -1.788654
12438    -8.862566  2.657612e-06    -1.785550
27513    -8.759241  3.486941e-06    -1.781266
4636     -8.750154  2.965013e-06    -1.780883
178      -8.725096  3.869058e-07    -1

In [95]:
# saving and viewing results
GSE151347_stats = GSE151347[["t-statistic","p-value","effect size"]]
GSE151347_stats = GSE151347_stats.dropna(axis=0, subset=["effect size"])
GSE151347_stats = GSE151347_stats.sort_values(by="effect size")

higher effect-size:
        t-statistic       p-value  effect size
9313      7.486681  1.160394e-05     1.716993
27907     7.550029  8.416382e-07     1.720778
21825     7.641057  1.612375e-05     1.726096
9289      7.699302  3.582986e-06     1.729424
12728     8.116543  4.187021e-06     1.751699
24831     8.492090  1.242135e-06     1.769612
6644      8.495601  9.495677e-07     1.769771
104       8.599307  2.068259e-06     1.774392
10870     8.859749  2.367085e-08     1.785435
23491     9.438096  1.990221e-06     1.807368 

lower effect-size:
        t-statistic       p-value  effect size
12406   -10.851753  5.371941e-08    -1.849130
12397   -10.669898  1.970289e-08    -1.844533
9932    -10.591128  4.137439e-07    -1.842479
20677    -9.105865  1.719878e-06    -1.795180
4803     -8.939265  2.624318e-06    -1.788654
12438    -8.862566  2.657612e-06    -1.785550
27513    -8.759241  3.486941e-06    -1.781266
4636     -8.750154  2.965013e-06    -1.780883
178      -8.725096  3.869058e-07    -

## Joining results

In [103]:
join_results_stats = GSE89775_stats.append([GSE104766_stats, GSE151347_stats])
join_results_stats = join_results_stats.dropna(axis=0, subset=["effect size"])
join_results_stats = join_results_stats.sort_values(by="effect size")
join_results_stats.to_csv("join_data_statistics.csv", index=False)

,t-statistic,p-value,effect size
RNU6-1054P,-22.047842,0.002051,-2.367142
AC008269.2,-39.963739,0.000013,-2.365151
RNU6-49P,-18.755730,0.002831,-2.364742
RP11-570H19.2,-18.755730,0.002831,-2.364742
AE000662.92,-18.755730,0.002831,-2.364742
RP11-472F19.1,-18.755730,0.002831,-2.364742
RP11-248E9.7,-18.755730,0.002831,-2.364742
RNU2-63P,-17.544339,0.002440,-2.359264
SCARNA1,-20.771381,0.000771,-2.356269
IGHV3-64,-24.901652,0.000156,-2.355964


## Joining the Datasets

In [ ]:
join_data = pd.concat([GSE89775, GSE104766, GSE151347], axis=1)
join_data.head()
stats_tests(join_data)

 40%|████      | 33510/83423 [10:46<21:04, 39.48it/s]

In [19]:
join_data_sats = join_data[["t-statistic","p-value","effect size"]]
join_data_sats = join_data_sats.dropna(axis=0, subset=["effect size"])
join_data_sats = join_data_sats.sort_values(by="effect size")

<class 'pandas.core.frame.DataFrame'>
Index: 52900 entries, 5S_rRNA to tRNA Pro
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   control  45000 non-null  float64
 1   control  45000 non-null  float64
 2   control  45000 non-null  float64
 3   case     45000 non-null  float64
 4   case     45000 non-null  float64
 5   case     45000 non-null  float64
 6   case     45000 non-null  float64
 7   case     45000 non-null  float64
 8   case     45000 non-null  float64
 9   case     45000 non-null  float64
 10  case     45000 non-null  float64
 11  case     45000 non-null  float64
 12  case     45000 non-null  float64
 13  control  28516 non-null  float64
 14  control  28516 non-null  float64
 15  control  28516 non-null  float64
 16  control  28516 non-null  float64
 17  control  28516 non-null  float64
 18  control  28516 non-null  float64
 19  control  28516 non-null  float64
 20  control  28516 non-null  float64
 21  control 

In [ ]:
join_data_sats.to_csv("join_data_statistics.csv", index=False)